# Elabastecedor

## Prepare

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

from datetime import date
from pathlib import Path

In [2]:
# Get datasets

path = Path("data")
entries = [path / entry for entry in os.listdir(path) if entry.endswith(".csv")]
entries.sort()
list(entries)

[PosixPath('data/20231231_products.csv'),
 PosixPath('data/20240104_products.csv'),
 PosixPath('data/20240107_products.csv'),
 PosixPath('data/20240111_products.csv'),
 PosixPath('data/20240114_products.csv'),
 PosixPath('data/20240117_products.csv'),
 PosixPath('data/20240121_products.csv'),
 PosixPath('data/20240124_products.csv'),
 PosixPath('data/20240128_products.csv'),
 PosixPath('data/20240131_products.csv'),
 PosixPath('data/20240204_products.csv'),
 PosixPath('data/20240207_products.csv'),
 PosixPath('data/20240211_products.csv'),
 PosixPath('data/20240214_products.csv'),
 PosixPath('data/20240218_products.csv'),
 PosixPath('data/20240221_products.csv'),
 PosixPath('data/20240225_products.csv'),
 PosixPath('data/20240228_products.csv'),
 PosixPath('data/20240303_products.csv'),
 PosixPath('data/20240306_products.csv'),
 PosixPath('data/20240310_products.csv'),
 PosixPath('data/20240313_products.csv'),
 PosixPath('data/20240317_products.csv'),
 PosixPath('data/20240320_products

In [3]:
# Load datasets

# Get date from filename
def get_date(filename):
    year = filename[0:4]
    month = filename[4:6]
    day = filename[6:8]
    return f"{year}-{month}-{day}"

# Append datasets in a list
dfs = []
for file in entries:
    date = get_date(file.name)
    df = pd.read_csv(file, index_col="id")
    df.drop_duplicates(inplace=True)
    df.rename(columns={"precio":date}, inplace=True)
    dfs.append(df)

len(dfs)

92

In [4]:
# Merge datasets

df = pd.concat(dfs)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 233838 entries, 44 to 61
Data columns (total 96 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   codigo      233838 non-null  int64  
 1   marca       233838 non-null  object 
 2   nombre      233838 non-null  object 
 3   2023-12-31  75 non-null      float64
 4   2024-01-04  73 non-null      float64
 5   2024-01-07  84 non-null      float64
 6   2024-01-11  3124 non-null    float64
 7   categoria   233606 non-null  object 
 8   2024-01-14  3103 non-null    float64
 9   2024-01-17  3033 non-null    float64
 10  2024-01-21  3067 non-null    float64
 11  2024-01-24  2978 non-null    float64
 12  2024-01-28  3034 non-null    float64
 13  2024-01-31  2946 non-null    float64
 14  2024-02-04  3082 non-null    float64
 15  2024-02-07  3062 non-null    float64
 16  2024-02-11  3102 non-null    float64
 17  2024-02-14  2988 non-null    float64
 18  2024-02-18  3080 non-null    float64
 19  2024-02-21

## Analyze

### Categories

In [5]:
# Get categories

categories = df[["codigo", "categoria"]].dropna().drop_duplicates(["codigo"]).set_index("codigo")
categories

,categoria
codigo,
114746,ACEITES
112222,ACEITES
103100,ACEITES
112537,ACEITES
114573,ACEITES
...,...
119965,BUDINES Y MAGDALENAS
103129,BUDINES Y MAGDALENAS
91000,HAMBURGUESAS Y MEDALLONES


In [6]:
categories.value_counts()

categoria                
GALLETITAS DULCES            171
YOGUR                        154
VINOS TINTOS                 136
LIMPIEZA DE COCINA Y BANO    121
CUIDADO DEL CABELLO          116
                            ... 
GRASAS                         2
GRANADINA                      2
LEVADURAS                      2
PESCADOS Y MARISCOS            2
SALSA GOLF                     2
Name: count, Length: 141, dtype: int64

### Products

In [7]:
# Get products

products = df[["codigo", "nombre", "marca"]].dropna().drop_duplicates(["codigo"]).set_index("codigo")
products["categoria"] = categories
products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4206 entries, 59 to 3015
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nombre     4206 non-null   object
 1   marca      4206 non-null   object
 2   categoria  4203 non-null   object
dtypes: object(3)
memory usage: 131.4+ KB


In [8]:
# Show products with missing category
products[products.categoria.isna()]

,nombre,marca,categoria
codigo,,,
27,BOLA DE LOMO XKG,A BALANZA,NaN
123525,TIERNITOS CARNE X1.5KG,INVENTARIABLES,NaN
123523,TIERNITOS CACHORROS C/L/C X1.5KG,EL ABASTECEDOR,NaN


In [9]:
# Drop products with missing category

products.dropna(inplace=True)
products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4203 entries, 59 to 3015
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   nombre     4203 non-null   object
 1   marca      4203 non-null   object
 2   categoria  4203 non-null   object
dtypes: object(3)
memory usage: 131.3+ KB


### Prices

In [10]:
# Get prices
prices = (df[df["codigo"].isin(categories.index)]
           .groupby("codigo")
           .sum()
           .replace(0, np.nan)
           .drop(columns=products.columns))
prices

,2023-12-31,2024-01-04,2024-01-07,2024-01-11,2024-01-14,2024-01-17,2024-01-21,2024-01-24,2024-01-28,2024-01-31,...,2024-10-13,2024-10-16,2024-10-20,2024-10-23,2024-10-27,2024-10-30,2024-11-03,2024-11-06,2024-11-10,2024-11-13
codigo,,,,,,,,,,,,,,,,,,,,,
2,5150.0,4099.0,4099.0,3699.0,NaN,3699.0,3699.0,3699.0,4849.0,5449.0,...,5899.0,5899.0,6199.0,6199.0,5899.0,5899.0,6199.0,6199.0,5899.0,5899.0
3,5600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5499.0,5499.0,7199.0,7199.0
4,5850.0,4799.0,4799.0,4399.0,4399.0,4399.0,4399.0,4399.0,4999.0,4999.0,...,6499.0,6499.0,5499.0,5499.0,6499.0,6499.0,6899.0,6899.0,6499.0,6499.0
7,7300.0,5899.0,5399.0,5399.0,5399.0,5399.0,5399.0,5399.0,6799.0,7299.0,...,7999.0,7999.0,8399.0,8399.0,NaN,7999.0,8399.0,8399.0,7999.0,7999.0
11,9800.0,7999.0,7999.0,7999.0,7999.0,7999.0,7999.0,NaN,7999.0,7999.0,...,9449.0,9449.0,7299.0,7299.0,9449.0,9449.0,9449.0,9449.0,NaN,9449.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150853,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1090.0,1090.0,1090.0,1090.0,1090.0,1090.0,1090.0,1200.0
150856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6199.0,6199.0,6199.0,6199.0,6199.0,6199.0
150889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,845.0,845.0,845.0,NaN,845.0,845.0


In [11]:
# Show prices' statistics per date
prices.describe()

,2023-12-31,2024-01-04,2024-01-07,2024-01-11,2024-01-14,2024-01-17,2024-01-21,2024-01-24,2024-01-28,2024-01-31,...,2024-10-13,2024-10-16,2024-10-20,2024-10-23,2024-10-27,2024-10-30,2024-11-03,2024-11-06,2024-11-10,2024-11-13
count,74.000000,73.000000,82.000000,3124.000000,3103.000000,3033.000000,3067.000000,2978.000000,3034.000000,2946.000000,...,2372.000000,2332.000000,2367.000000,2358.000000,2413.000000,2381.000000,2426.000000,2396.000000,2468.000000,2409.000000
mean,2999.040541,2872.684932,2488.621951,2496.435915,2530.728073,2621.589149,2618.719677,2688.513227,2632.066882,2671.545815,...,3782.235919,3804.600600,3781.043629,3791.761934,3765.307082,3789.277190,3816.177894,3826.926674,3821.175976,3832.833093
std,2793.892931,2641.349344,2474.422560,3496.646721,3448.710021,3734.677710,3655.598256,3700.222567,3641.347483,3677.161676,...,5722.266292,5758.878928,5722.021621,5731.457508,5660.825056,5719.982701,5740.365569,5769.555951,5711.647280,5784.985990
min,90.000000,90.000000,120.000000,59.990000,59.990000,59.990000,110.000000,110.000000,110.000000,120.000000,...,126.990000,126.990000,126.990000,126.990000,126.990000,126.990000,126.990000,126.990000,126.990000,126.990000
25%,724.000000,799.000000,711.500000,987.250000,999.000000,1030.000000,1027.500000,1079.000000,1045.000000,1069.000000,...,1430.000000,1423.500000,1437.500000,1439.250000,1425.000000,1425.000000,1425.000000,1440.000000,1449.000000,1449.000000
50%,1599.500000,1719.000000,1345.000000,1536.000000,1599.000000,1650.000000,1690.000000,1725.500000,1710.000000,1749.000000,...,2282.500000,2299.000000,2295.000000,2299.000000,2299.000000,2299.000000,2299.000000,2300.000000,2299.000000,2299.000000
75%,5112.250000,4799.000000,4146.500000,2700.000000,2822.000000,2930.000000,2889.000000,2990.000000,2839.000000,2915.000000,...,3999.000000,3999.000000,3999.000000,3999.000000,3945.000000,3955.000000,4000.000000,3999.000000,4000.000000,3999.000000
max,9999.000000,9999.000000,12000.000000,71415.000000,71415.000000,90483.000000,79839.000000,79839.000000,79839.000000,79839.000000,...,129999.000000,129999.000000,129999.000000,129999.000000,129999.000000,129999.000000,129999.000000,129999.000000,129999.000000,129999.000000


In [12]:
# Show price's statistics per product
prices_per_product = prices.T
prices_per_product.describe()

codigo,2,3,4,7,11,12,13,15,16,17,...,150829,150835,150842,150844,150847,150853,150856,150889,150925,150931
count,90.000000,11.000000,91.000000,88.000000,87.000000,78.000000,21.0,88.000000,85.000000,90.000000,...,2.0,8.0,8.0,8.0,9.0,8.000000,6.0,5.0,2.0,2.000000
mean,5753.455556,6253.636364,6245.714286,7727.420455,8863.379310,6390.025641,11349.0,3081.954545,2313.117647,6506.800000,...,999.0,4490.0,1765.0,3789.0,899.0,1103.750000,6199.0,845.0,1299.0,2439.500000
std,593.191845,623.375051,771.601643,784.168946,742.336134,692.480183,0.0,927.637165,557.022899,781.822463,...,0.0,0.0,0.0,0.0,0.0,38.890873,0.0,0.0,0.0,84.145707
min,3699.000000,5499.000000,4399.000000,5399.000000,6999.000000,4999.000000,11349.0,1599.000000,1399.000000,4299.000000,...,999.0,4490.0,1765.0,3789.0,899.0,1090.000000,6199.0,845.0,1299.0,2380.000000
25%,5649.000000,5799.500000,5999.000000,7374.250000,8399.000000,5999.000000,11349.0,2299.000000,1999.000000,6049.000000,...,999.0,4490.0,1765.0,3789.0,899.0,1090.000000,6199.0,845.0,1299.0,2409.750000
50%,5899.000000,6199.000000,6499.000000,7999.000000,9449.000000,6899.000000,11349.0,2999.000000,2199.000000,6499.000000,...,999.0,4490.0,1765.0,3789.0,899.0,1090.000000,6199.0,845.0,1299.0,2439.500000
75%,6199.000000,6699.000000,6899.000000,7999.000000,9449.000000,6899.000000,11349.0,4199.000000,2999.000000,7199.000000,...,999.0,4490.0,1765.0,3789.0,899.0,1090.000000,6199.0,845.0,1299.0,2469.250000
max,6199.000000,7199.000000,6899.000000,8399.000000,9800.000000,6899.000000,11349.0,4199.000000,2999.000000,7199.000000,...,999.0,4490.0,1765.0,3789.0,899.0,1200.000000,6199.0,845.0,1299.0,2499.000000


In [13]:
# price change per date
change_per_date = prices_per_product.pct_change(fill_method=None) * 100
change_per_date

codigo,2,3,4,7,11,12,13,15,16,17,...,150829,150835,150842,150844,150847,150853,150856,150889,150925,150931
2023-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-04,-20.407767,NaN,-17.965812,-19.191781,-18.377551,NaN,NaN,NaN,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-07,0.000000,NaN,0.000000,-8.476013,0.000000,NaN,NaN,NaN,NaN,-14.303571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-11,-9.758478,NaN,-8.335070,0.000000,0.000000,NaN,NaN,NaN,NaN,-10.418837,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-14,NaN,NaN,0.000000,0.000000,0.000000,0.0,NaN,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-30,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000,0.000000,0.000000,...,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
2024-11-03,5.085608,NaN,6.154793,5.000625,0.000000,NaN,NaN,68.027211,50.025013,0.000000,...,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
2024-11-06,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,0.000000,...,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN
2024-11-10,-4.839490,30.914712,-5.797942,-4.762472,NaN,NaN,NaN,0.000000,0.000000,30.914712,...,NaN,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN


In [14]:
# Show prices' statistics per date
change_per_date.T.describe()

,2023-12-31,2024-01-04,2024-01-07,2024-01-11,2024-01-14,2024-01-17,2024-01-21,2024-01-24,2024-01-28,2024-01-31,...,2024-10-13,2024-10-16,2024-10-20,2024-10-23,2024-10-27,2024-10-30,2024-11-03,2024-11-06,2024-11-10,2024-11-13
count,0.0,67.000000,53.000000,78.000000,3049.000000,3002.000000,2948.000000,2969.000000,2901.000000,2922.000000,...,2286.000000,2309.000000,2284.000000,2315.000000,2330.000000,2371.000000,2348.000000,2367.000000,2371.000000,2390.000000
mean,NaN,3.378456,-0.785705,8.438657,3.891930,2.225437,2.088244,2.612687,-0.185708,0.781613,...,0.176806,0.592999,0.852061,0.348690,0.494281,0.155329,0.676078,0.313529,0.839783,-0.003076
std,NaN,16.987697,17.008046,22.316726,16.749631,10.859529,14.541099,10.953936,10.865165,8.816803,...,8.608734,6.799962,9.742348,4.238803,9.807264,3.511615,10.558948,3.967689,7.705256,3.942960
min,NaN,-20.407767,-42.424242,-42.892857,-42.074776,-40.016006,-50.050000,-35.714286,-64.428571,-63.660899,...,-54.206219,-42.768496,-39.299750,-43.314438,-56.312195,-41.025641,-57.514379,-57.171429,-42.857143,-53.400000
25%,NaN,0.000000,-7.846154,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,0.000000,0.000000,17.809986,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,NaN,55.833333,61.616162,105.444444,174.413457,124.068966,353.805073,177.570093,168.168168,204.554697,...,126.859504,94.444444,118.370264,70.972424,125.000000,59.980000,138.722644,45.859873,100.000000,41.701418


In [15]:
def pct_change(key, values, df):
    "Group by dataset, get pct_change from mean"
    return (df
     .assign(**{key: values})
     .groupby(key)
     .mean()
     .pct_change(fill_method=None) * 100)

In [16]:
# change per month

key = "Mes"
values = [column[0:7] for column in prices.columns]
change_per_month = pct_change(key, values, prices_per_product)
change_per_month

codigo,2,3,4,7,11,12,13,15,16,17,...,150829,150835,150842,150844,150847,150853,150856,150889,150925,150931
Mes,,,,,,,,,,,,,,,,,,,,,
2023-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01,-19.194175,NaN,-21.004748,-20.257230,-18.377551,NaN,NaN,NaN,NaN,-13.904762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02,33.191157,NaN,19.535476,24.527113,3.125391,10.502100,NaN,13.571915,12.385951,23.907633,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-03,2.217612,NaN,9.403411,3.602029,4.512331,10.610266,NaN,13.668372,14.003463,7.672135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-04,4.339001,NaN,7.744847,5.178204,5.541880,6.569257,NaN,30.398646,15.186707,6.866352,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-05,-1.903070,NaN,-1.535744,2.391301,-6.105677,1.855691,0.0,6.196884,2.326664,3.636893,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06,4.215286,NaN,1.364735,1.099037,1.162360,-2.010353,0.0,0.000000,0.000000,1.052779,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07,1.332941,NaN,3.077397,-0.077649,9.328628,6.154793,NaN,35.011671,31.832651,-6.482382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08,0.000000,NaN,2.985520,-0.874233,0.000000,0.000000,NaN,3.704618,3.449465,3.960984,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Show prices' statistics per month
change_per_month.T.describe()

Mes,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11
count,0.0,72.000000,3116.000000,3165.000000,3054.000000,2845.000000,2652.000000,2548.000000,2524.000000,2541.000000,2526.000000,2436.000000
mean,NaN,9.666119,8.348681,6.641454,5.329718,2.399522,2.514801,2.727310,3.276916,3.489117,2.312708,1.686112
std,NaN,27.031032,21.276038,13.862161,12.220355,11.753905,10.050030,9.409572,11.462903,10.061676,9.489604,8.262957
min,NaN,-42.546784,-60.025447,-47.826087,-37.061256,-60.534195,-41.818182,-73.179104,-65.006906,-59.452177,-62.707345,-54.580153
25%,NaN,-9.326777,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,0.490870,3.544087,3.592085,2.009306,0.000000,0.682136,1.254216,1.791534,1.912617,1.141957,0.000000
75%,NaN,29.171531,12.872996,10.809463,9.239517,4.575163,5.155639,6.128217,5.698692,6.493238,4.870251,4.272780
max,NaN,87.945434,275.000000,168.000000,150.000000,258.333333,190.019002,123.027068,300.667780,113.952325,137.603306,81.340670


In [18]:
# change per quarter

def calc_quarter(date):
    y = date[0:4]
    m = date[5:7]
    q = ((int(m) - 1) // 3) + 1
    return f"{y}-{q}"

key = "Trimestre"
values = [calc_quarter(date) for date in prices.columns]
change_per_quarter = pct_change(key, values, prices_per_product)
change_per_quarter

codigo,2,3,4,7,11,12,13,15,16,17,...,150829,150835,150842,150844,150847,150853,150856,150889,150925,150931
Trimestre,,,,,,,,,,,,,,,,,,,,,
2023-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-1,-0.097087,12.482143,-7.840894,-6.230769,-15.275510,NaN,NaN,NaN,NaN,2.386676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-2,15.028781,NaN,20.100732,17.755402,5.419728,16.947241,NaN,52.086437,30.691686,23.324501,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-3,2.919272,NaN,5.241461,1.097464,7.499334,5.519461,0.0,40.943401,35.955399,-3.337576,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-4,-1.636715,NaN,-5.982819,-0.055779,-3.387564,0.000000,0.0,-17.745492,-15.773509,-6.266389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Show prices' statistics per quarter
change_per_quarter.T.describe()

Trimestre,2023-4,2024-1,2024-2,2024-3,2024-4
count,0.0,73.000000,3200.000000,2733.000000,2578.000000
mean,NaN,38.316990,14.339887,8.316215,6.149102
std,NaN,56.364365,24.413136,13.752499,13.708022
min,NaN,-39.742105,-62.812400,-47.122152,-65.428440
25%,NaN,-0.647929,1.718361,1.097464,0.525895
50%,NaN,18.049410,9.595403,7.290880,5.247917
75%,NaN,61.807808,22.579153,13.836056,10.519834
max,NaN,223.645546,389.141068,152.732910,239.051868


In [20]:
# change per year

key = "Año"
values = [date[0:4] for date in prices.columns]
change_per_year = pct_change(key, values, prices_per_product)
change_per_year

codigo,2,3,4,7,11,12,13,15,16,17,...,150829,150835,150842,150844,150847,150853,150856,150889,150925,150931
Año,,,,,,,,,,,,,,,,,,,,,
2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024,11.849242,12.839286,6.839506,5.922374,-9.668486,NaN,NaN,NaN,NaN,16.374799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Show prices' statistics per year
change_per_year.T.describe()

Año,2023,2024
count,0.0,74.000000
mean,NaN,90.892787
std,NaN,122.609159
min,NaN,-12.873882
25%,NaN,13.067468
50%,NaN,46.788987
75%,NaN,115.898631
max,NaN,656.363636


In [22]:
pct_change("Categoria", categories, change_per_month.T)

Mes,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11
Categoria,,,,,,,,,,,,
ACCESORIOS DE VIDRIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ACCESORIOS PARA EL MATE,NaN,NaN,236.088869,68.967625,9.002949,-73.617666,-544.321279,22.965741,-164.468911,-28.466043,-895.391940,-182.720575
ACCESORIOS PLÁSTICOS,NaN,NaN,-144.926255,-72.726939,-35.586974,-205.251644,-94.338798,-100.000000,-100.000000,-111.878937,-122.061226,615.394155
ACEITES,NaN,NaN,-13.930119,112.442143,364.665976,-284.773127,13.917131,-inf,inf,-298.629013,-253.989630,-68.405430
ACHURAS,NaN,NaN,-100.000000,223.298439,-78.915865,-1951.526090,-2233.726836,-805.365763,330.457995,-100.000000,749.320615,449.147223
...,...,...,...,...,...,...,...,...,...,...,...,...
VINOS ROSADOS,NaN,NaN,-52.498974,121.558727,5.249388,49.547565,1073.747329,94.399812,58.894679,-101.900663,-15.699303,73.719695
VINOS TINTOS,NaN,NaN,202.557523,-26.680058,-40.803011,-48.199411,-69.222911,-35.383061,-42.721571,-5104.959024,7.991760,-58.951295
WHISKYS,NaN,NaN,-25.914787,274.245007,145.203277,-59.746319,-152.183231,-29.626682,11.295998,85.668040,41.222655,-71.497685


In [23]:
pct_change("Categoria", categories, change_per_quarter.T)

Trimestre,2023-4,2024-1,2024-2,2024-3,2024-4
Categoria,,,,,
ACCESORIOS DE VIDRIO,NaN,NaN,NaN,NaN,NaN
ACCESORIOS PARA EL MATE,NaN,NaN,111.094032,-297.277935,87.382802
ACCESORIOS PLÁSTICOS,NaN,NaN,-93.292909,-76.826739,-93.594829
ACEITES,NaN,NaN,1112.326759,-285.536278,544.242710
ACHURAS,NaN,NaN,62.753776,976.470798,618.164450
...,...,...,...,...,...
VINOS ROSADOS,NaN,NaN,24.126982,84.922473,-6.770209
VINOS TINTOS,NaN,NaN,-38.512138,-37.860112,-3.809819
WHISKYS,NaN,NaN,96.842971,-23.944429,37.241378


In [24]:
pct_change("Categoria", categories, change_per_year.T)

Año,2023,2024
Categoria,,
ACCESORIOS DE VIDRIO,NaN,NaN
ACCESORIOS PARA EL MATE,NaN,NaN
ACCESORIOS PLÁSTICOS,NaN,NaN
ACEITES,NaN,NaN
ACHURAS,NaN,NaN
...,...,...
VINOS ROSADOS,NaN,NaN
VINOS TINTOS,NaN,NaN
WHISKYS,NaN,NaN
